In [2]:
from omegaconf import OmegaConf as om
from omegaconf import DictConfig
from typing import cast
import sys 
import os
sys.path.append(os.path.abspath(".."))

from main import build_model
import src.create_bert as bert_module
import src.create_model as model_module
from transformers import AutoTokenizer, Trainer, TrainingArguments



yaml_path = "../yamls/pretrain/micro_dna_monarch-mixer-pretrain-786dim-80m-parameters.yaml"

tokenizer = AutoTokenizer.from_pretrained("gagneurlab/SpeciesLM", revision="downstream_species_lm")


with open(yaml_path) as f:
    cfg = om.load(f)
cfg = cast(DictConfig, cfg)
print(cfg.max_duration)
model = model_module.create_model(cfg.model.get("model_config"))
#model = bert_module.create_bert_mlm(model_config = cfg.model.get("model_config", None))

Could not import FlashFFTConv!


/opt/modules/i12g/anaconda/envs/m2-mixer/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: libtorch_cuda_cu.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")


200000ba
Using Monarch Mixer for Sequence Mixing: True
-- Bidirectional: True
-- Using Long Conv Residual: True
-- Hyena w: 10
-- Hyena w mod: 1
-- Hyena filter order: 128
-- Hyena filter dropout: 0.2
-- Hyena filter wd: 0.1
-- Hyena filter emb dim: 5
-- Hyena filter lr: 0.001
-- Hyena filter lr pos emb: 1e-05


In [2]:
model

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(5504, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): MonarchMixerSequenceMixing(
            (filter_fn): HyenaFilter(
              (dropout): Dropout(p=0.2, inplace=False)
              (pos_emb): PositionalEmbedding()
              (implicit_filter): Sequential(
                (0): Linear(in_features=5, out_features=128, bias=True)
                (1): Sin()
                (2): Linear(in_features=128, out_features=128, bias=True)
                (3): Sin()
                (4): Linear(in_features=128, out_features=128, bias=True)
                (5): Sin()
                (6)

In [41]:
sequence = "TGTAAATATT"

# Break down the sequence into 6-mers
six_mers = [sequence[i:i+6] for i in range(0, len(sequence), 1)]

# Encode each 6-mer
encoded_sequence = [tokenizer.encode(six_mer) for six_mer in six_mers]

encoded = tokenizer.convert_tokens_to_ids(six_mers)

print(six_mers)
print(encoded_sequence)

['TGTAAA', 'GTAAAT', 'TAAATA', 'AAATAT', 'AATATT', 'ATATT', 'TATT', 'ATT', 'TT', 'T']
[[2, 1861, 3], [2, 3334, 3], [2, 1033, 3], [2, 22, 3], [2, 74, 3], [2, 1, 3], [2, 1, 3], [2, 1, 3], [2, 1, 3], [2, 1, 3]]


In [28]:
vocab = tokenizer.get_vocab()

for token, id in vocab.items():
    print(f"Token: {token}, ID: {id}")


Token: TCACAC, ID: 1575
Token: TTCCTA, ID: 1449
Token: ACGTGA, ID: 737
Token: TACCGG, ID: 1204
Token: CGAACC, ID: 2831
Token: CGAGGA, ID: 2881
Token: metschnikowia_bicuspidata_var_bicuspidata_nrrl_yb_4993_gca_001664035, ID: 5209
Token: CGAGAC, ID: 2871
Token: AGATTC, ID: 795
Token: wolfiporia_cocos_md_104_ss10_gca_000344635, ID: 5171
Token: CTGGAC, ID: 2551
Token: GGCTCG, ID: 4000
Token: aspergillus_avenaceus_gca_009193465, ID: 5235
Token: CATGCC, ID: 2175
Token: AGCTGC, ID: 931
Token: CTGCAG, ID: 2536
Token: ATCGAT, ID: 438
Token: GATTGA, ID: 3169
Token: TTCTAC, ID: 1431
Token: TGTAAG, ID: 1864
Token: ACTTCC, ID: 607
Token: TAGCGC, ID: 1267
Token: TAGCAG, ID: 1256
Token: TCCCAC, ID: 1703
Token: thermothielavioides_terrestris_gca_900343105, ID: 4466
Token: CGGCTG, ID: 3052
Token: fusarium_euwallaceae_gca_003957675, ID: 4118
Token: GAGTGT, ID: 3298
Token: CCTTTA, ID: 2649
Token: blumeria_graminis_f_sp_tritici_gca_900519115, ID: 5303
Token: ACTAGA, ID: 593
Token: TCAGCA, ID: 1597
Token: 

In [27]:
from datasets import load_from_disk
from collate import DataCollatorForLanguageModelingSpan
from torch.utils.data import DataLoader

dataset = load_from_disk("../micro")
dataset = dataset.remove_columns(["species_name", "__index_level_0__"])

data_collator = DataCollatorForLanguageModelingSpan(tokenizer, mlm=True, mlm_probability = 0.02, span_length = 6)

train_dataloader = DataLoader(dataset["train"], batch_size=1, shuffle=True, collate_fn=data_collator)
sample = next(iter(train_dataloader))
sample

{'input_ids': tensor([[   2, 4203, 1030,   12,   33,  120,  466, 1850, 3292,  865, 3445, 1478,
         1804, 3108,  129,  502, 1994, 3868, 3169,  373, 1477, 1797, 3078,    9,
           22,   74,  281, 1112,  337, 1334, 1226,  794, 3161,  343, 1357, 1317,
         1160,  531, 2112,  242,  953, 3798, 2889, 3349, 1096,  276, 1089,  246,
          971, 3870, 3179,  416, 1650, 2492, 1761, 2934,    4,    4,    4,    4,
            4,    4,    4,    4,    4,    4,    4,    4, 1370, 1371, 1375, 1390,
         1450, 1691, 2655, 2414, 1451, 1694, 2666, 2459, 1630, 2410, 1434, 1627,
         2398, 1386, 1434, 1626, 2395, 1374, 1388, 1444, 1668, 2561, 2037, 4039,
         3854, 3113,  150,  586, 2330, 1116,  353, 1400, 1491, 1856, 3314,  956,
         3810, 2939, 3549, 1894, 3465, 1558, 2121,  280, 1108,  324, 1282,    4,
            4,    4,    4,    4,    4, 3471, 1581, 2216,  660, 2628, 2306, 1018,
         4060, 3940, 3457, 1528, 2002, 3898, 3289,  853, 3398,    4,    4,    4,
            4,

In [3]:
from datasets import load_from_disk
from collate import DataCollatorForLanguageModelingSpan

#Load model directly
%env WANDB_PROJECT=singlesamplednam2



dataset = load_from_disk("../micro")
dataset = dataset.remove_columns(["species_name", "__index_level_0__"])

data_collator = DataCollatorForLanguageModelingSpan(tokenizer, mlm=True, mlm_probability = 0.02, span_length = 6)

training_args = TrainingArguments(
    output_dir="./results/correct_model_1sample",
    
    max_steps=100000,
    
    seed=17,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=1,
    
    logging_strategy="steps",
    logging_steps=1,
    
    remove_unused_columns=False,
    evaluation_strategy="no",
    
    #dataloader_num_workers=0,
    run_name="repo_model_1sample",
    report_to="wandb"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    data_collator=data_collator
)


#trainer.train()
tokenizer.vocab_size

env: WANDB_PROJECT=singlesamplednam2


4101

In [2]:
model

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(4096, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): MonarchMixerSequenceMixing(
            (filter_fn): HyenaFilter(
              (dropout): Dropout(p=0.2, inplace=False)
              (pos_emb): PositionalEmbedding()
              (implicit_filter): Sequential(
                (0): Linear(in_features=5, out_features=128, bias=True)
                (1): Sin()
                (2): Linear(in_features=128, out_features=128, bias=True)
                (3): Sin()
                (4): Linear(in_features=128, out_features=128, bias=True)
                (5): Sin()
                (6): Linear

In [31]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(dataset["train"], batch_size=1, shuffle=True, collate_fn=data_collator)
for step, inputs in enumerate(train_dataloader):
    inputs = {
        "args": inputs
    }
    model(**inputs)

TypeError: HuggingFaceModel.forward() got an unexpected keyword argument 'args'